In [ ]:
G = G.to_undirected()

# Get node attributes for authors and their flair
authors = list(G.nodes())
flairs = nx.get_node_attributes(G, 'flair')

# Dataframe with authors and flairs
df_flairs = pd.DataFrame(list(flairs.items()), columns=['Author', 'Flair'])

trials = 100
number_of_communities = np.zeros(trials)
for i in range(trials):
    if i % 100 == 0:
        print(i)
    partition = community.best_partition(G)
    number_of_communities[i] = len(set(partition.values()))

med = int(np.median(number_of_communities))
# Print the number of communities
print("Mean number of communities:", np.mean(number_of_communities))
print("Median number of communities:", int(np.median(number_of_communities)))

In [ ]:
def modularity(G,partition):
    modularity_vals = []
    L = G.number_of_edges()
    for i in range(len(partition)):
        Lc = G.subgraph(partition[i]).number_of_edges()
        kc = 0
        for node in partition[i]:
            kc += G.degree(node)
        modularity_vals.append(Lc/L - (kc/(2*L))**2)
    return sum(modularity_vals)

while True:
    partition = community.best_partition(G)
    if len(set(partition.values())) == med:
        break

# Create dataframe with author and community
df_community = pd.DataFrame(list(partition.items()),columns = ['Author','Community'])
df_community = df_community.sort_values(by='Community')

In [ ]:
# Save df_community as pkl
with open('LargeData/df_community.pickle', 'wb') as f:
    pickle.dump(df_community, f)

In [ ]:
# modularity
print('Modularity of best partition:',modularity(G, [df_community[df_community['Community']==i]['Author'].values for i in range(len(df_community['Community'].unique()))]))

In [ ]:
number_of_unique_communities = len(df_community['Community'].unique())
number_of_members_in_each_community = df_community['Community'].value_counts().values

labels = ['Community ' + str(i) + '\n' + str(number_of_members_in_each_community[i]) + ' members' for i in range(number_of_unique_communities)]
plt.figure(figsize=(12,7))
squarify.plot(sizes=df_community['Community'].value_counts().values,label=labels,alpha=.8,color=sb.color_palette("Set3", number_of_unique_communities),ec='black')
plt.title('Communities')
plt.axis('off')
plt.show()

In [ ]:
# find the authors from the graph in data
authors = list(G.nodes())
flairs = nx.get_node_attributes(G,'flair')

# Make dataframe with authors and flairs
df_flairs = pd.DataFrame(list(flairs.items()),columns = ['Author','Flair'])

# Compute modularity for the authors divided into flairs
modularity_flairs = modularity(G, [df_flairs[df_flairs['Flair']==i]['Author'].values for i in df_flairs['Flair'].unique()])
print('Modularity Flairs:',modularity_flairs)